In [67]:
# import sys
# !{sys.executable} -mpip install -U jupyter


In [68]:
import os
import re

import numpy as np
import openai
import pandas as pd
from tqdm.notebook import tqdm


In [69]:
(gc1_df := pd.read_csv("data/GC1.csv"))[:3]


,Party,Message,review_id,democrat,topic,chatID,depolarization_index_change,affect_index_change,other_party_voters_change,other_party_voters_change_shortterm,timestamp
0,R,"I live in STATE1, we have very strict gun laws...",NaN,1.0,gunControl,3tgMqapg9a80CtK5IEPJ,1.207007,0.153148,0.26,0.34,0.29
1,D,I grew up in Idaho where hunting was common an...,NaN,0.0,gunControl,3tgMqapg9a80CtK5IEPJ,0.468500,0.044630,0.44,0.72,0.21
2,R,"The guns we had were rifles since my father, m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
gc1_df.iloc[1]["Message"]


'I grew up in Idaho where hunting was common and no problems arose from gun ownership for hunting. No one felt they needed a gun for self protection. In the 20 or so years since it seems non-hunters have guns for self defense. Not for hunting. I don’t own a gun and don’t feel I need one for self-defense.'

In [71]:
# openai.api_key = os.environ["OPENAI_API_KEY"]

with open("OPENAI_API_KEY.txt", "r") as f:
    openai.api_key = f.read().strip()


In [72]:
# Different warnings for different DVs.

# Conversation again
# warning_string = ("This might be a counterproductive thing to say, if these "
# "two ever hope to talk again. ")

# More sympathetic to each other's point of view
warning_string = (
    "This kind of statement might be harmful. "
    # "This might be a counterproductive thing to say, if these "
    # "two are to ever see each others' points of view. "
)

# Non-violence
# warning_string = ("This might be a counterproductive thing to say, if we want "
#                   "these two to not support political violence or a civil war in the next decade. ")


In [73]:
rephrasing_specs = {
    "validate": lambda party, opp_party: (
        warning_string
        + f"It is usually more helpful to first validate or affirm what the "
        f"other person thinks. "
        f"Can you suggest a rephrasing where the {party} first "
        f"affirms or validates what the {opp_party} said?"
    ),
    # "paraphrase_and_ask": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     f"paraphrases what the {opp_party} said last, and then asks a clarifying "
    #     f"question or question of understanding? (NOT a 'trap' question or a "
    #     f"question to make a point)"
    # ),
    # "rephrase_to_i": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     "makes a statement beginning with 'I feel that...' or 'It seems to me that' "
    #     "instead of making braod, generalizing claims?"
    # ),
    # "restate_before_disagree": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     f"restates, in their own words, the {opp_party}'s last message before "
    #     "introducing disagreement?"
    # ),
    # "empathy": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     f"emphasizes shared beliefs and values with the {opp_party}, "
    #     "essentially indicating that we all care about the same big-picture things "
    #     "rather than emphasizing the disagreement in the details?"
    # ),
    # "vulnerability": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     "emphasizes self-disclosure, ",
    #     "personal stories, humor, and social trust as a way to improve their conversation "
    #     "with the {opp_party} and decision-making? The idea is that vulnerability increases "
    #     "trust and understanding among others, which seems crucial when having "
    #     "disagreements and discussions.",
    # ),
    # "translate_values": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} "
    #     "still communicates their position, but motivates it with different "
    #     "underlying concerns? It also doesn't necessarily have to emphasize that "
    #     "we all believe the same thing."
    # ),
    # "perspective_taking": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     f"expresses what they think it would be like to be in {opp_party}'s "
    #     "position, and then reaffirm the original message?"
    # ),
    # "perspective_getting": lambda party, opp_party: (
    #     warning_string
    #     + f"Can you suggest a rephrasing of that message, where the {party} instead "
    #     f"actively requests the view of {opp_party} in a non-judgmental and "
    #     "listening way?"
    # ),
}


In [74]:
exemplar_running_convo = [
    "Republican: I think the current gun control laws do not need any further regulation as it will only restrict the rights of law abiding citizens and leave them more vulnerable to criminals that avert gun control laws anyway. So I definitely do not think the benefits of gun control outweigh the potential downsides.",
    "Democrat: I think there should be stricter background checks, not only the mentally ill but also people with misdemeanor charges, especially if it is some sort of violence; and longer wait times. There also need to be background checks at gun shows. I believe All guns need to be registered.",
    "Republican: Gun ownership already requires registration of the firearm(s), FYI.",
]


In [75]:
EXEMPLAR_PROMPT = '''"""
{instructions}

{exemplar_running_convo}

Now the Republican wants to say: '{republican_message}'

{rephrasing_spec}

Here is the rephrasing:
"{exemplar_rephrasing}"
"""'''


In [76]:
PROMPT = '''"""
{instructions}

{running_convo}

Now the {party} wants to say: '{message}'.

{rephrasing_spec}

Here is the rephrasing:
"'''


In [77]:
def create_rephrasings(df, *, dry_run=False, n_turns=2, force_recreate=False):
    if os.path.exists("rephrasings.csv") and not force_recreate:
        raise FileExistsError(
            "rephrasings.csv already exists. Use force_recreate=True to overwrite."
        )

    parties = []
    messages = []

    running_convo = []
    instructions = (
        "The following is an exchange between two people who strongly disagree about"
        " gun control in the United States."
    )

    for theory, spec in tqdm(rephrasing_specs.items(), desc="theories"):
        rephrasings = []
        prompts = []
        for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="rows"):
            party = {"D": "Democrat", "R": "Republican"}[row.Party]
            opp_party = {"D": "Republican", "R": "Democrat"}[row.Party]
            message = row.Message
            parties.append(party)
            if len(running_convo) >= n_turns + 1:
                running_convo = running_convo[1:]
            running_convo.append(f"{party}: {message}")

            if len(running_convo) == 1:
                prompts.append(None)
                messages.append(None)
                rephrasings.append(None)
                continue

            messages.append(message)
            republican_message = re.search(
                "(?<=Republican: ).*", exemplar_running_convo[-1]
            ).group(0)
            exemplar_rephrasing = (
                "I understand that you would feel safer if all guns in the United"
                " States were registered. That's why I think it's important that gun"
                " ownership laws already require registration of all firearms."
            )

            exemplar_prompt = EXEMPLAR_PROMPT.format(
                instructions=instructions,
                exemplar_running_convo="\n".join(exemplar_running_convo[:-1]),
                republican_message=republican_message,
                rephrasing_spec=spec("Republican", "Democrat"),
                exemplar_rephrasing=exemplar_rephrasing,
            )

            prompt = PROMPT.format(
                instructions=instructions,
                running_convo="\n".join(running_convo[:-1]),
                party=party,
                message=message,
                rephrasing_spec=spec(party, opp_party),
            )
            full_prompt = f"{exemplar_prompt}\n\n{prompt}"

            if not dry_run:
                response = openai.Completion.create(
                    engine="davinci", prompt=full_prompt, max_tokens=100
                )

                rephrasing = response["choices"][0]["text"]
                rephrasings.append(rephrasing)
            else:
                rephrasings.append(None)
            prompts.append(full_prompt)

        if not dry_run:
            df[f"{theory}_rephrasing"] = rephrasings
        df[f"{theory}_prompt"] = prompts
    df.to_csv("rephrasings.csv")


In [60]:
create_rephrasings(pd.read_csv("data/GC1.csv")[:5], dry_run=False, force_recreate=True)


theories:   0%|          | 0/1 [00:00<?, ?it/s]

rows:   0%|          | 0/5 [00:00<?, ?it/s]

In [55]:
def latexify_rephrasings(df):
    with pd.option_context("max_colwidth", None):
        with open("rephrasings.txt", "w") as f:
            for theory, spec in rephrasing_specs.items():
                subset = f"Party Message {theory}_rephrasing".split()
                f.write(
                    df[subset].to_latex(
                        multirow=True,
                        longtable=True,
                        column_format="p{0.05\\textwidth}|p{0.05\\textwidth}|p{0.45\\textwidth}|p{0.45\\textwidth}",
                    )
                )
                f.write("%==============================================\n")


In [61]:
(rephrasings_df := pd.read_csv("rephrasings.csv"))[:3]


,Unnamed: 0,Party,Message,review_id,democrat,topic,chatID,depolarization_index_change,affect_index_change,other_party_voters_change,other_party_voters_change_shortterm,timestamp,validate_rephrasing,validate_prompt
0,0,R,"I live in STATE1, we have very strict gun laws...",NaN,1.0,gunControl,3tgMqapg9a80CtK5IEPJ,1.207007,0.153148,0.26,0.34,0.29,NaN,NaN
1,1,D,I grew up in Idaho where hunting was common an...,NaN,0.0,gunControl,3tgMqapg9a80CtK5IEPJ,0.468500,0.044630,0.44,0.72,0.21,I understand you enjoyed hunting and collectin...,"""""""\nThe following is an exchange between two ..."
2,2,R,"The guns we had were rifles since my father, m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"You lived in a state where hunting was common,...","""""""\nThe following is an exchange between two ..."


In [64]:
print(rephrasings_df.iloc[1]["validate_prompt"])


"""
The following is an exchange between two people who strongly disagree about gun control in the United States.

Republican: I think the current gun control laws do not need any further regulation as it will only restrict the rights of law abiding citizens and leave them more vulnerable to criminals that avert gun control laws anyway. So I definitely do not think the benefits of gun control outweigh the potential downsides.
Democrat: I think there should be stricter background checks, not only the mentally ill but also people with misdemeanor charges, especially if it is some sort of violence; and longer wait times. There also need to be background checks at gun shows. I believe All guns need to be registered.

Now the Republican wants to say: 'Gun ownership already requires registration of the firearm(s), FYI.'

This kind of statement might be harmful. It is usually more helpful to first validate or affirm what the other person thinks. Can you suggest a rephrasing where the Republic

In [65]:
print(rephrasings_df.iloc[1]["validate_rephrasing"])


I understand you enjoyed hunting and collecting guns when you were a child. I also understand that you think that owning guns is a family tradition, and thus you feel that guns are very important. It can be important since you seem to know that many teenagers, who might not own firearms, spend time at your home. This is why I think that, from your experience, you would feel that it is safer when people have guns. This is not my experience and I do not agree."
B.
